# Wasserstein GAN - Improved

ref.:
GULRAJANI, Ishaan et al.  
Improved training of wasserstein gans.  
In: Advances in neural information processing systems. 2017. p. 5767-5777.

![new objective](wgan_gp_objective.png)

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

Using TensorFlow backend.


In [3]:
# auxiliary class

In [4]:
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

## Generator

In [5]:
def build_generator(latent_dim, channels):

    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(Reshape((7, 7, 128)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=4, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=4, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

## Critic

In [6]:
def build_critic(img_shape):

    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

## helper

In [7]:
def gradient_penalty_loss(y_true, y_pred, averaged_samples):
    """
    Computes gradient penalty based on prediction and weighted real / fake samples
    """
    gradients = K.gradients(y_pred, averaged_samples)[0]
    # compute the euclidean norm by squaring ...
    gradients_sqr = K.square(gradients)
    #   ... summing over the rows ...
    gradients_sqr_sum = K.sum(gradients_sqr,
                              axis=np.arange(1, len(gradients_sqr.shape)))
    #   ... and sqrt
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    # compute lambda * (1 - ||grad||)^2 still for each single sample
    gradient_penalty = K.square(1 - gradient_l2_norm)
    # return the mean as loss over all the batch samples
    return K.mean(gradient_penalty)

In [8]:
def wasserstein_loss(y_true, y_pred):
    """ same loss as in WGAN """
    return K.mean(y_true * y_pred)

In [9]:
def sample_images(G, latent_dim, epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

## train

In [10]:
def train(G, critic_model, generator_model, 
          n_critic, latent_dim,
          epochs, batch_size, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = -np.ones((batch_size, 1))
    fake =  np.ones((batch_size, 1))
    dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
    for epoch in range(epochs):

        for _ in range(n_critic):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            # Sample generator input
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            # Train the critic
            d_loss = critic_model.train_on_batch([imgs, noise],
                                                 [valid, fake, dummy])

        # ---------------------
        #  Train Generator
        # ---------------------

        g_loss = generator_model.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f] [G loss: %f]" % (epoch, d_loss[0], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0 or epoch == epochs - 1:
            sample_images(G, latent_dim, epoch)

# main()

In [11]:
if not os.path.exists('images'):
    os.makedirs('images')

In [12]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [13]:
# Following parameter and optimizer set as recommended in paper
n_critic = 5

In [14]:
# create optimizer
optimizer = RMSprop(lr=0.00005)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Build the generator model
G = build_generator(latent_dim, channels)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)      

In [16]:
# Build the critic model
C = build_critic(img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)         

In [17]:
#-------------------------------
# Construct Computational Graph
#       for the Critic
#-------------------------------

In [18]:
# Freeze generator's layers while training critic
G.trainable = False

In [19]:
# Image input (real sample)
real_img = Input(shape=img_shape)

In [20]:
# Noise input
z_disc = Input(shape=(latent_dim,))

# Generate image based of noise (fake sample)
fake_img = G(z_disc)

In [21]:
# Discriminator determines validity of the real and fake images
fake = C(fake_img)
valid = C(real_img)

In [22]:
# Construct weighted average between real and fake images
interpolated_img = RandomWeightedAverage()([real_img, fake_img])

In [23]:
# Determine validity of weighted sample
validity_interpolated = C(interpolated_img)

In [24]:
# Use Python partial to provide loss function with additional
# 'averaged_samples' argument
partial_gp_loss = partial(gradient_penalty_loss,
                          averaged_samples=interpolated_img)

partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

critic_model = Model(inputs=[real_img, z_disc],
                     outputs=[valid, fake, validity_interpolated])

critic_model.compile(loss=[wasserstein_loss,
                           wasserstein_loss,
                           partial_gp_loss],
                     optimizer=optimizer,
                     loss_weights=[1, 1, 10])

In [25]:
#-------------------------------
# Construct Computational Graph
#         for Generator
#-------------------------------

In [26]:
# For the generator we freeze the critic's layers
C.trainable = False
G.trainable = True  # this is the default

In [27]:
# Sampled noise for input to generator
z_gen = Input(shape=(latent_dim,))

In [28]:
# Generate images based of noise
img = G(z_gen)

In [29]:
# Discriminator determines validity
valid = C(img)

In [30]:
# Defines generator model
generator_model = Model(z_gen, valid)
generator_model.compile(loss=wasserstein_loss, optimizer=optimizer)

In [31]:
# train

In [33]:
# epochs=30000
epochs=3000
train(G, critic_model, generator_model, 
      n_critic, latent_dim,
      epochs=epochs, batch_size=32, sample_interval=100)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 25.070177] [G loss: 0.297224]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

2221 [D loss: -1.130304] [G loss: -1.807348]
2222 [D loss: 0.090930] [G loss: -1.769228]
2223 [D loss: -0.852175] [G loss: -2.471143]
2224 [D loss: -0.314273] [G loss: -1.639558]
2225 [D loss: -0.589768] [G loss: -1.708480]
2226 [D loss: -0.320955] [G loss: -2.343223]
2227 [D loss: -0.678880] [G loss: -1.910802]
2228 [D loss: -1.190974] [G loss: -1.877266]
2229 [D loss: -0.236641] [G loss: -2.085944]
2230 [D loss: -0.443879] [G loss: -2.287195]
2231 [D loss: -0.377624] [G loss: -2.311561]
2232 [D loss: -0.411889] [G loss: -2.082436]
2233 [D loss: 0.307874] [G loss: -2.138365]
2234 [D loss: -0.312012] [G loss: -2.197287]
2235 [D loss: -0.420102] [G loss: -1.980470]
2236 [D loss: -0.530272] [G loss: -2.048275]
2237 [D loss: -0.643914] [G loss: -1.814736]
2238 [D loss: -0.823666] [G loss: -2.131377]
2239 [D loss: -0.783549] [G loss: -2.109231]
2240 [D loss: -0.861815] [G loss: -2.039255]
2241 [D loss: -0.344496] [G loss: -1.940282]
2242 [D loss: -0.350523] [G loss: -1.784209]
2243 [D loss

2404 [D loss: -1.020910] [G loss: -1.913471]
2405 [D loss: -0.276055] [G loss: -1.998328]
2406 [D loss: -0.790286] [G loss: -2.327264]
2407 [D loss: -0.131145] [G loss: -2.652147]
2408 [D loss: -0.606227] [G loss: -2.310210]
2409 [D loss: -0.156687] [G loss: -1.813769]
2410 [D loss: -0.230035] [G loss: -1.695987]
2411 [D loss: -0.354228] [G loss: -1.760433]
2412 [D loss: -0.926099] [G loss: -2.032331]
2413 [D loss: -0.678029] [G loss: -1.797156]
2414 [D loss: -0.792520] [G loss: -1.678052]
2415 [D loss: -1.130708] [G loss: -1.893006]
2416 [D loss: -0.709074] [G loss: -1.607876]
2417 [D loss: -0.689505] [G loss: -2.328692]
2418 [D loss: -0.745283] [G loss: -1.848363]
2419 [D loss: -1.100448] [G loss: -2.041864]
2420 [D loss: -0.476270] [G loss: -1.897526]
2421 [D loss: -0.159874] [G loss: -2.060741]
2422 [D loss: -0.649533] [G loss: -2.559935]
2423 [D loss: -0.074863] [G loss: -2.467247]
2424 [D loss: -0.634030] [G loss: -1.756130]
2425 [D loss: -0.142192] [G loss: -2.650424]
2426 [D lo

2587 [D loss: -0.022238] [G loss: -3.427532]
2588 [D loss: -1.135002] [G loss: -2.551661]
2589 [D loss: -0.895322] [G loss: -2.826949]
2590 [D loss: -0.735255] [G loss: -2.528460]
2591 [D loss: -0.464322] [G loss: -3.087500]
2592 [D loss: -1.002025] [G loss: -2.577790]
2593 [D loss: -0.295367] [G loss: -2.780842]
2594 [D loss: -0.383757] [G loss: -2.628974]
2595 [D loss: 0.091811] [G loss: -2.634712]
2596 [D loss: -0.155601] [G loss: -2.216531]
2597 [D loss: -0.757965] [G loss: -2.599074]
2598 [D loss: -0.389670] [G loss: -2.233825]
2599 [D loss: -0.158039] [G loss: -2.048759]
2600 [D loss: -0.207427] [G loss: -2.743587]
2601 [D loss: -0.949204] [G loss: -2.818963]
2602 [D loss: -1.128345] [G loss: -1.793543]
2603 [D loss: -0.816962] [G loss: -2.938653]
2604 [D loss: -0.522919] [G loss: -2.487713]
2605 [D loss: -0.640048] [G loss: -2.302669]
2606 [D loss: -0.916172] [G loss: -2.558714]
2607 [D loss: 0.783831] [G loss: -2.546304]
2608 [D loss: -0.422204] [G loss: -1.916369]
2609 [D loss

2770 [D loss: -0.515736] [G loss: -1.808630]
2771 [D loss: -0.229639] [G loss: -1.334187]
2772 [D loss: -1.272943] [G loss: -1.539988]
2773 [D loss: -0.678001] [G loss: -2.090042]
2774 [D loss: 0.508894] [G loss: -1.816962]
2775 [D loss: -0.269314] [G loss: -2.091750]
2776 [D loss: -0.293400] [G loss: -2.043352]
2777 [D loss: -0.211729] [G loss: -2.206542]
2778 [D loss: -0.564505] [G loss: -1.974314]
2779 [D loss: -0.315506] [G loss: -1.895327]
2780 [D loss: -0.486703] [G loss: -1.655749]
2781 [D loss: -0.621419] [G loss: -1.806374]
2782 [D loss: -0.914662] [G loss: -1.582647]
2783 [D loss: -0.575426] [G loss: -1.838942]
2784 [D loss: -0.347804] [G loss: -1.639193]
2785 [D loss: -0.303407] [G loss: -1.819773]
2786 [D loss: 0.164593] [G loss: -1.517576]
2787 [D loss: 0.046832] [G loss: -1.470986]
2788 [D loss: -0.551222] [G loss: -1.384371]
2789 [D loss: -0.070516] [G loss: -1.922161]
2790 [D loss: -0.482492] [G loss: -1.887630]
2791 [D loss: -0.132974] [G loss: -1.777997]
2792 [D loss:

2953 [D loss: -0.905535] [G loss: -1.091216]
2954 [D loss: -0.549705] [G loss: -1.695293]
2955 [D loss: 0.094844] [G loss: -1.161019]
2956 [D loss: -0.966454] [G loss: -0.985748]
2957 [D loss: -0.745350] [G loss: -1.092926]
2958 [D loss: -0.684335] [G loss: -1.239940]
2959 [D loss: -0.959831] [G loss: -1.067470]
2960 [D loss: -1.015069] [G loss: -1.386908]
2961 [D loss: -1.111937] [G loss: -1.353749]
2962 [D loss: -0.330204] [G loss: -1.453471]
2963 [D loss: 0.075954] [G loss: -1.356759]
2964 [D loss: -0.258247] [G loss: -1.669227]
2965 [D loss: -0.163486] [G loss: -1.205422]
2966 [D loss: -1.168602] [G loss: -1.359461]
2967 [D loss: -0.563122] [G loss: -0.920528]
2968 [D loss: -1.211767] [G loss: -1.456300]
2969 [D loss: -0.387015] [G loss: -1.049794]
2970 [D loss: -0.642056] [G loss: -1.412410]
2971 [D loss: -0.505135] [G loss: -1.436659]
2972 [D loss: -0.590590] [G loss: -1.478811]
2973 [D loss: -0.506580] [G loss: -1.328103]
2974 [D loss: -0.704124] [G loss: -1.559672]
2975 [D loss